In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
%matplotlib inline
import pickle
import re
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import LabelEncoder
stop_words = set(stopwords.words('english'))
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from sklearn.naive_bayes import GaussianNB

In [2]:
# nltk.download('stopwords')
# nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\91973\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\91973\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
df = pd.read_csv('Tweets.csv')

In [4]:
# df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [5]:
# df.columns

Index(['tweet_id', 'airline_sentiment', 'airline_sentiment_confidence',
       'negativereason', 'negativereason_confidence', 'airline',
       'airline_sentiment_gold', 'name', 'negativereason_gold',
       'retweet_count', 'text', 'tweet_coord', 'tweet_created',
       'tweet_location', 'user_timezone'],
      dtype='object')

In [6]:
df = df[['text', 'airline_sentiment']]

In [7]:
# df.head()

,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,neutral
1,@VirginAmerica plus you've added commercials t...,positive
2,@VirginAmerica I didn't today... Must mean I n...,neutral
3,@VirginAmerica it's really aggressive to blast...,negative
4,@VirginAmerica and it's a really big bad thing...,negative


In [9]:
def regex_standardize(text):
    url_pattern = r'\b(?:https?://|www\:.)[^\s]+\b' 
    text = re.sub(url_pattern, '', text)
    username_tag_pattern = r'@\S+'
    text = re.sub(username_tag_pattern, '', text)
    punc_removing = r'[^a-z A-Z 0-9]'
    text = re.sub(punc_removing, '', text)
    return text

In [37]:
# df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   text               14640 non-null  object
 1   airline_sentiment  14640 non-null  object
dtypes: object(2)
memory usage: 228.9+ KB


In [10]:
df.isnull().sum()

text                 0
airline_sentiment    0
dtype: int64

In [40]:
df.shape

(14640, 2)

In [11]:
df["text"] = df["text"].apply(lambda x: regex_standardize(x))

### Text Preprocessing

In [16]:
def preprocessing_pipeline(text):
    # text = " ".join([word for word in text.split() if word not in stop_words])
    text = " ".join([lemmatizer.lemmatize(word, pos='v') for word in text.split()])
    return text

In [17]:
df["text"] = df["text"].apply(lambda x: preprocessing_pipeline(x))

In [18]:
%%time
pretrained_model = SentenceTransformer('distilbert/distilbert-base-uncased-finetuned-sst-2-english')

d:\Projects\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


No sentence-transformers model found with name distilbert/distilbert-base-uncased-finetuned-sst-2-english. Creating a new one with mean pooling.


In [19]:
def bert_embedding(text):
    return pretrained_model.encode(text)

In [20]:
%%time
df['bert_embeding'] = df['text'].apply(lambda x: bert_embedding(x))

CPU times: total: 39min 37s
Wall time: 8min 5s


In [21]:
label_encoder = LabelEncoder()

In [22]:
df['label']=label_encoder.fit_transform(df['airline_sentiment'])

In [24]:
df["airline_sentiment"].value_counts()

airline_sentiment
negative    9178
neutral     3099
positive    2363
Name: count, dtype: int64

In [25]:
df["label"].value_counts()

label
0    9178
1    3099
2    2363
Name: count, dtype: int64

In [26]:
X = list(df["bert_embeding"])
y = df["label"]

In [27]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train, y_test = train_test_split(X,y,train_size=0.2, random_state=42, shuffle=True)

In [28]:
len(X_train), len(y_train)

(2928, 2928)

In [29]:
from sklearn.naive_bayes import GaussianNB
naive_bayes_gau_model = GaussianNB()
naive_bayes_gau_model.fit(X_train, y_train)

GaussianNB()

In [30]:
y_pred = naive_bayes_gau_model.predict(X_test)

In [31]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

In [32]:
class_report = classification_report(y_test, y_pred)
acc_score = accuracy_score(y_test, y_pred)
confusion_matrix = confusion_matrix(y_test, y_pred)

In [33]:
print(class_report)
print(acc_score)
print(confusion_matrix)

              precision    recall  f1-score   support

           0       0.81      0.86      0.83      7332
           1       0.42      0.27      0.33      2482
           2       0.59      0.73      0.66      1898

    accuracy                           0.71     11712
   macro avg       0.61      0.62      0.61     11712
weighted avg       0.69      0.71      0.70     11712

0.7141393442622951
[[6288  690  354]
 [1200  681  601]
 [ 255  248 1395]]


In [34]:
from sklearn.ensemble import RandomForestClassifier
rfc_model = RandomForestClassifier()

In [115]:
# from sklearn.model_selection import StratifiedKFold
# cv = StratifiedKFold()

In [125]:
# criterion = ['gini', 'entropy', 'log_loss']
# max_features = ['sqrt', 'log2', None]
# #n_estimators = [50, 100, 150]

In [126]:
# # params = dict(criterion=criterion, max_features= max_features, n_estimators=n_estimators)
# params = dict(criterion=criterion, max_features= max_features) #, n_estimators=n_estimators)

In [127]:
# params

{'criterion': ['gini', 'entropy', 'log_loss'],
 'max_features': ['sqrt', 'log2', None]}

In [122]:
# from sklearn.model_selection import RandomizedSearchCV

In [128]:
# random_search = RandomizedSearchCV(rfc_model, param_distributions=params, cv=cv,random_state=42)

In [35]:
# random_search.fit(X_train,y_train)

In [36]:
rfc_model.fit(X_train, y_train)

RandomForestClassifier()

In [37]:
y_pred_rfc = rfc_model.predict(X_test)

In [39]:
class_report = classification_report(y_test, y_pred_rfc)
acc_score = accuracy_score(y_test, y_pred_rfc)
# confusion_matrix = confusion_matrix(y_test, y_pred_rfc)
print(class_report)
print(acc_score)
# print(confusion_matrix)

              precision    recall  f1-score   support

           0       0.79      0.94      0.86      7332
           1       0.67      0.35      0.46      2482
           2       0.74      0.66      0.70      1898

    accuracy                           0.77     11712
   macro avg       0.73      0.65      0.67     11712
weighted avg       0.76      0.77      0.75     11712

0.7716017759562842


## Testing model with manual text

In [40]:
user_input = input("Check your emotions: ")
text = regex_standardize(user_input)
text = preprocessing_pipeline(text)
text = bert_embedding(text)
predicted_emotions = rfc_model.predict([text])
emotion_decoded = label_encoder.inverse_transform(predicted_emotions)
print(f"your emotion seems to be: {emotion_decoded[0]}")

your emotion seems to be: positive


In [43]:
with open("random_search_model.pkl", "wb") as file:
  pickle.dump(rfc_model, file)

with open("label_encoder.pkl", "wb") as file:
  pickle.dump(label_encoder, file)